## chapter 3 test

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
graph=tf.Graph()

全局风格的Variable对象(一般是模型中的超参)，本例中是global_step即模型训练参数和total_output即模型输出累加和

In [3]:
with graph.as_default():
    with tf.name_scope("variables"):
        #记录数据流图运行次数的变量和累加和的变量
        global_step=tf.Variable(0,dtype=tf.int32,trainable=False,name="global_step")
        total_output=tf.Variable(0.0,dtype=tf.float32,trainable=False,name="total_output")
    with tf.name_scope("transformation"):
        with tf.name_scope("input"):
            #占位符
            a=tf.placeholder(tf.float32,shape=[None],name="input_placeholder_a")
        with tf.name_scope("intermediate_layer"):
            b=tf.reduce_prod(a,name="product_b")
            c=tf.reduce_sum(a,name="sum_c")
        with tf.name_scope("output"):
            output=tf.add(b,c,name="output")
        with tf.name_scope("update"):
            update_total=total_output.assign_add(output)
            increment_step=global_step.assign_add(1)
        with tf.name_scope("summaries"):
            avg=tf.div(update_total,tf.cast(increment_step,tf.float32),name="average")
            tf.summary.scalar('Output',output)
            tf.summary.scalar('Sum_of_outputs_over_time',update_total)
            tf.summary.scalar('Average_of_outputs_over_time',avg)
        with tf.name_scope("global_ops"):
            init=tf.global_variables_initializer()
            merged_summaries=tf.summary.merge_all()

In [4]:
sess=tf.Session(graph=graph)
writer=tf.summary.FileWriter("./chapter3_test",sess.graph)
sess.run(init)

变换测试长度

In [5]:
def run_graph(input_tensor):
    feed_ditc={a:input_tensor}
    _,step,summary=sess.run([output,increment_step,merged_summaries],feed_dict=feed_ditc)
    writer.add_summary(summary,global_step=step)

In [6]:
run_graph([2.0,8.0])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])

In [7]:
writer.flush()
writer.close()
sess.close()